In [ ]:
from Bio import Entrez
from time import sleep

from entrez import get_publication_count, assign_publication_index
from interactome import get_kegg_pathways, get_string_interactors, assign_interactome_index 
from uniprot import get_human_uniprot_id, get_uniprot_length, get_alphafold_prediction, get_uniprot_3d, get_uniprot_disease 
from clinvar import fetch_variant_ids, split_variation_name, fetch_variant_details


def main():
    
    gene_name = input("Enter gene name: ")
    email = input("Enter a valid email address: ")

    if not email or not gene_name:
        print("Please provide both gene name and email address.")
    else:
        Entrez.email = email 
            
        uniprot_id = get_human_uniprot_id(gene_name)
        print(f"UniProt ID for {gene_name}: {uniprot_id}")
        print()

        if uniprot_id:
            pub_count = get_publication_count(gene_name)
            print(f"Number of publications about {gene_name} from 2000 to present: {pub_count}")
            print()

            pub_index = assign_publication_index(pub_count)
            print(f"Publication index for {gene_name}: {pub_index}")
            print()

            pathways_count = get_kegg_pathways(gene_name)
            print(f"Number of pathways {gene_name} is involved in: {pathways_count}")
            print()

            interactors_count = get_string_interactors(gene_name)
            print(f"Number of {gene_name} interactors from the STRING-DB: {interactors_count}")
            print()

            interactome_index = assign_interactome_index(pathways_count, interactors_count)
            print(f"Interactome index for {gene_name}: {interactome_index}")
            print()

            uniprot_prot_length = get_uniprot_length(uniprot_id)    
            print(f"Length of {gene_name}: {uniprot_prot_length}")
            print()

            uniprot_3d_struct, structure_index = get_uniprot_3d(uniprot_id)
            print(f"3D structure information for {gene_name}:\n")
            for structure in uniprot_3d_struct:
                print(structure)
                print()
                
            print(f"Structure index for {gene_name}: {structure_index}")
            print()

            uniprot_pathology = get_uniprot_disease(uniprot_id)
            print(f"Involvement of {gene_name} in diseases:\n")
            diseases = uniprot_pathology.split("\n")
            for disease in diseases:
                print(disease)
                print()

            search_api_url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=clinvar&term={gene_name}[gene]+AND+single_gene[prop]+AND+missense_variant[molcons]&retmax=900'
            variant_ids = fetch_variant_ids(search_api_url)
            print(f"List of missense mutations for {gene_name} from ClinVar:")
            sleep(5)
            print("This operation might take a while. We suggest you to grab a coffee :)")
            if variant_ids:
                df_all_variants = fetch_variant_details(variant_ids)
                if not df_all_variants.empty:                    
                    display(df_all_variants)
                else:
                    print("No variant details available.")
            else:
                print("No variant IDs found.")

if __name__ == "__main__":
    main()